# Results of LeNet Trained on MNIST

In this notebook I display results in terms of accuracy on test data for LeNet models trained on the MNIST dataset using various regularization techniques. The LeNet model and the MNIST dataset were set up as in Hoffman 2019 (for all the models). The MNIST data is preprocessed by normalizing using mean 0.1307 and variance 0.3081. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization, both with and without dropout with a dropout rate of p_drop = 0.5. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 250 epochs.

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import pickle
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_MNIST
from model_classes import LeNet_MNIST
from plotting_tools import get_random_img, generate_random_vectors
from tools import accuracy, compute_total_variation, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_MNIST()

# Summary of model
summary_model = LeNet_MNIST().to(device)
summary(summary_model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
           Dropout-6                  [-1, 120]               0
            Linear-7                   [-1, 84]          10,164
           Dropout-8                   [-1, 84]               0
            Linear-9                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
---------------------------------------------

In [5]:
# Load models
dataset = "mnist"

model_names_set = [
    "model_no_reg",
    "model_no_reg_no_dropout",
    "model_l2",
    "model_l2_no_dropout",
    "model_jacobi",
    "model_jacobi_no_dropout",
    "model_svb",
    "model_svb_no_dropout",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

model_names.append("model_jacobi_and_svb_0")
model_names.append("model_all_0")
model_names.append("model_jacobi_and_l2_0")
model_names.append("model_svb_and_l2_0")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (%)
0,no_regdropout,98.684 +/- 0.074
1,no_reg_no_dropout,98.872 +/- 0.088
2,l2dropout,99.144 +/- 0.087
3,l2_no_dropout,99.162 +/- 0.059
4,jacobidropout,98.652 +/- 0.111
5,jacobi_no_dropout,98.846 +/- 0.087
6,svbdropout,98.416 +/- 0.082
7,svb_no_dropout,98.716 +/- 0.052


### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [6]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Define the models
model_names = [
    "model_no_reg_0",
    "model_no_reg_no_dropout_0",
    "model_l2_0",
    "model_l2_no_dropout_0",
    "model_jacobi_0",
    "model_jacobi_no_dropout_0",
    "model_svb_0",
    "model_svb_no_dropout_0",
    "model_jacobi_and_svb_0",
    "model_all_0",
    "model_jacobi_and_l2_0",
    "model_svb_and_l2_0",
]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

100%|██████████| 12/12 [00:00<00:00, 68.61it/s]


In [7]:
print("Isotropic Total variation for three different zoom levels (of images generated)")
display(df_results_isotropic)

Isotropic Total variation for three different zoom levels (of images generated)


0.025                       0.010  \
                                   mean conf_interval          mean   
model_no_reg_0              9644.814889   7761.946029  21856.282118   
model_no_reg_no_dropout_0   2652.066605    2532.25229  10848.329613   
model_l2_0                  2868.740017   1014.000593   3959.517678   
model_l2_no_dropout_0       1288.871235   1367.890078   4199.804319   
model_jacobi_0             11131.085316   3626.156139  16084.572622   
model_jacobi_no_dropout_0   6311.785583   2429.540256     6112.0129   
model_svb_0                 2402.258516   1352.377712   3455.646171   
model_svb_no_dropout_0      1289.459135   1134.754337   3058.661244   
model_jacobi_and_svb_0      2988.592093   1509.480684   3472.244655   
model_all_0                 2179.105652   1069.341392   2531.907065   
model_jacobi_and_l2_0        2650.94444    978.902572   1195.778898   
model_svb_and_l2_0          2332.192878   1078.378443   3040.417619   

                                                0.001                
                          conf_interval          mean conf_interval  
model_no_reg_0              6679.083854  34514.446799   4719.105455  
model_no_reg_no_dropout_0   2069.054594  30895.399605   4696.195096  
model_l2_0                  1043.800228   7651.990257    2415.33475  
model_l2_no_dropout_0       1246.433071   8891.442107    1824.81879  
model_jacobi_0              3701.962555   9999.305969   2671.378385  
model_jacobi_no_dropout_0   3210.434625   2469.842697   2166.486604  
model_svb_0                  759.956723   3418.508243   1701.446704  
model_svb_no_dropout_0      1397.945627   9464.586826   2130.350315  
model_jacobi_and_svb_0       936.450916   7323.622669   2296.099979  
model_all_0                 1100.802708   3051.648024    821.946702  
model_jacobi_and_l2_0        393.012649    231.480285    205.941821  
model_svb_and_l2_0          1002.600316   5024.826718   1424.531837

#### Anisotropic Total Variation

In [12]:
# Define zoom levels, number of images, and confidence level for confidence interval calculation
zoom_levels = [0.025, 0.01, 0.001]
n_images = 10
confidence_level = 0.95

# Select only the first model of each type (those with index 0)
selected_models = [name for name in model_names if name.split("_")[-1] == "0"]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=selected_models, columns=cols)

# Loop over the selected models
for name in tqdm(selected_models):
    model = models[name].model  # Get model
    model.to(device)
    tv_values = {
        zoom: [] for zoom in zoom_levels
    }  # To store total variation values for each zoom level

    # Generate tv values for n_images number of images
    for _ in range(n_images):
        # Get random image and vectors
        img = get_random_img(test_loader)
        v1, v2 = generate_random_vectors(img)

        # Compute the isotropic total variation of decision boundaries for the current model
        # and the generated image at different zoom levels
        tv_list = compute_total_variation(
            model,
            img,
            v1,
            v2,
            device,
            resolution=300,
            zoom=zoom_levels,
            mode="anisotropic",
        )
        for zoom, tv in zip(zoom_levels, tv_list):
            tv_values[zoom].append(tv)

    # Calculate mean and 95% confidence interval for total variation values at each zoom level
    for zoom in zoom_levels:
        mean = np.mean(tv_values[zoom])
        std_err = np.std(tv_values[zoom]) / np.sqrt(n_images)
        conf_interval = stats.t.ppf((1 + confidence_level) / 2, n_images - 1) * std_err
        df_results_anisotropic.loc[name, (zoom, "mean")] = mean
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = conf_interval

100%|██████████| 8/8 [19:16<00:00, 144.58s/it]


In [14]:
print(
    "Anisotropic Total variation for three different zoom levels (of images generated)"
)
display(df_results_anisotropic)

Anisotropic Total variation for three different zoom levels (of images generated)


0.025                  0.010                \
                              mean conf_interval     mean conf_interval   
model_no_reg_0             16489.0   9579.953016  30759.8   8207.474962   
model_no_reg_no_dropout_0   3665.3   2524.393369  10611.8   2894.782772   
model_l2_0                  3113.4   1205.333345   4432.4    804.842024   
model_l2_no_dropout_0       2711.6    1535.62887   5235.4   1014.917218   
model_jacobi_0              6393.5    4817.27876  12939.2   7291.401038   
model_jacobi_no_dropout_0   7213.8   1951.732378   8802.0   2345.853662   
model_svb_0                 2080.0   1569.950794   2607.2   1023.666724   
model_svb_no_dropout_0      2244.1   1333.621615   5279.2   1569.307022   

                             0.001                
                              mean conf_interval  
model_no_reg_0             38131.4   5955.673933  
model_no_reg_no_dropout_0  31105.9   6065.683073  
model_l2_0                 10451.5    1249.21686  
model_l2_no_dropout_0      11885.9   2469.744112  
model_jacobi_0             12374.6   4050.805704  
model_jacobi_no_dropout_0   6056.3   4895.328621  
model_svb_0                 2889.3    1631.19518  
model_svb_no_dropout_0     12895.4   3264.318483